<html>
<table width="100%" cellspacing="2" cellpadding="2" border="1">
<tbody>
<tr>
<td valign="center" align="center" width="25%"><img src="media/decartes.jpg"
alt="DeCART Icon" width="128" height="171"><br>
</td>
<td valign="center" align="center" width="75%">
<h1 align="center"><font size="+1">DeCART Summer School<br>
for<br>
Biomedical Data Science</font></h1></td>
<td valign="center" align="center" width="25%"><img
src="media/U_Health_stacked_png_red.png" alt="Utah Health
Logo" width="128" height="134"><br>
</td>
</tr>
</tbody>
</table>
<br>
</html>


# Working with Textual Data
#### &copy; Brian E. Chapman, Ph.D.


### To get the latest version of this lesson, execute the cell below

In [1]:
%matplotlib inline

In [2]:
import pymysql
import pandas as pd
import getpass
from textblob import TextBlob

In [4]:
conn = pymysql.connect(host="mysql",
                       port=3306,user="jovyan",
                       passwd=getpass.getpass("Enter MySQL passwd for jovyan"),db='mimic2')
cursor = conn.cursor()

Enter MySQL passwd for jovyan········


### Use Pandas and SQL to create a dataframe with the following:
* subject_id
* hospital admission id
* text of the radiology report
* Limit the number of reports to 10000

In [5]:
rad_data = \
pd.read_sql("""SELECT noteevents.subject_id, 
                      noteevents.hadm_id,
                      noteevents.text 
               FROM noteevents
               WHERE noteevents.category = 'RADIOLOGY_REPORT' LIMIT 10000""",conn)
rad_data.head(5)

,subject_id,hadm_id,text
0,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...
1,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...
2,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:43 AM\n ...
3,56,28766.0,\n\n\n DATE: [**2644-1-17**] 6:37 AM\n ...
4,56,28766.0,\n\n\n DATE: [**2644-1-19**] 12:09 PM\n ...


In [6]:
rad_data.shape

(10000, 3)

## Section Splitting

<img src="http://medicine.utah.edu/dbmi/images/faculty/Chapman,Wendy_240x360.jpg"
alt="Wendy Chapman" width="128">

Clinical reports are often divided into sections. Physicians are taught to write their reports in the SOAP format: subjective information, objective information, assessment, and plan, and sections often reflect this process. Information each setion may be formatted differently. For example, subjective information is often narrative, telling a story of the patient's description of their past medical history and their current issues. The objective section may be much more telegraphic with shortened descriptions of measurements like blood pressure and heart rate, such as "120/80 88". You may need different NLP techniques to accurately extract information out of different sections. Sentence splitting, for example, may be different in subjective versus objective sections. Radiology reports are not in the SOAP format, but section identification can still be very important, because in these reports the radiologist describes what she sees on the radiographic image then sometimes summarizes her impression of what the observations may represent, such as "the infiltrate could be consistent with pneumonia." For some tasks, you may only be interested in the impression, which has more weight than the reasoning that occurs in the Findings section.

## Exercise

Write a function that returns the impression section of a report. The function will take two arguments:

1. The text to search for the impression section is the first positional argument.
1. A list of phrases to use to identify the impression section.

Not every report will have a section labeled "IMPRESSION". You can consider "INTERPRETATION" and "CONCLUSIONS" as synonyms for "IMPRESSION"

If you cannot identify the impresison section (or equivalent), return an empty string.

#### Hints

* Use a for loop  to iterate across
* Use the find method of a string to identify where in the string the impression section occurs
* Use slicing to grab the impression section


In [29]:

def find_impression(txt, search_terms=["IMPRESSION", "INTERPRETATION", "CONCLUSIONS"]):
    
    for term in search_terms:
        ind = txt.find(term)
        if ind != -1:
            return txt[ind:]
    return " "

find_impression(txt10)


'IMPRESSION:\n     1. Persistence of interval slight worsening of degree of patchy opacity within\n     the lingula.  Continued follow up is recommended to exclude pneumonia.\n     2. Persistent mild CHF.\n\n'

In [26]:
txt10 = rad_data.loc[10, "text"]
txt1 = rad_data.loc[1, "text"]
print(txt10)    




     DATE: [**3264-8-16**] 1:53 PM
     CHEST (PORTABLE AP)                                             Clip # [**Clip Number (Radiology) 10701**]
     Reason: 68 yo M CHF, SOB                                            
     Admitting Diagnosis: CONGESTIVE HEART FAILURE
     ______________________________________________________________________________
     UNDERLYING MEDICAL CONDITION:
        68 year old man with sob, hx of chf, cad s/p CABG                             
       
     REASON FOR THIS EXAMINATION:
      68 yo M CHF, SOB                                                                
     ______________________________________________________________________________
                                     FINAL REPORT
     INDICATION:  Shortness of breath with history of CHF and CAD.
     
     Comparison [**3264-8-14**].
     
     FINDINGS:  Patient is status post CABG and multiple sternal wires appear
     fractured, not changed since the previous exam.  Cardiac and 

## Exercise 

Use ``apply()`` to create a new column in the ``rad_data`` named "IMPRESSIONS" with the string identified with ``find_impression()``.

In [30]:
rad_data["IMPRESSIONS"] = rad_data["text"].apply(find_impression)

rad_data.head()

,subject_id,hadm_id,text,IMPRESSIONS
0,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...,
1,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:53 AM\n ...,
2,56,28766.0,\n\n\n DATE: [**2644-1-17**] 10:43 AM\n ...,IMPRESSION: Stable appearance of right pariet...
3,56,28766.0,\n\n\n DATE: [**2644-1-17**] 6:37 AM\n ...,IMPRESSION:\n \n Cardiomegaly and mild...
4,56,28766.0,\n\n\n DATE: [**2644-1-19**] 12:09 PM\n ...,IMPRESSION:\n \n Marked improvement in...


In [54]:
import numpy as np
def do_math(func):
    return[func(i) for i in np.arange(-np.pi, np.pi, 0.1)]

In [55]:
do_math(np.cos)

[-1.0,
 -0.9950041652780257,
 -0.9800665778412416,
 -0.9553364891256059,
 -0.9210609940028849,
 -0.8775825618903724,
 -0.8253356149096779,
 -0.7648421872844879,
 -0.6967067093471648,
 -0.6216099682706637,
 -0.5403023058681389,
 -0.4535961214255764,
 -0.36235775447667246,
 -0.2674988286245862,
 -0.1699671429002396,
 -0.07073720166770146,
 0.02919952230129027,
 0.1288444942955263,
 0.22720209469308872,
 0.32328956686350513,
 0.4161468365471441,
 0.5048461045998591,
 0.5885011172553474,
 0.6662760212798258,
 0.737393715541247,
 0.8011436155469351,
 0.8568887533689485,
 0.9040721420170622,
 0.9422223406686591,
 0.9709581651495912,
 0.9899924966004459,
 0.9991351502732796,
 0.9982947757947529,
 0.9874797699088644,
 0.9667981925794602,
 0.9364566872907952,
 0.8967584163341455,
 0.8481000317104064,
 0.7909677119144145,
 0.7259323042001377,
 0.6536436208636092,
 0.5748239465332662,
 0.4902608213406962,
 0.4007991720799713,
 0.30733286997841586,
 0.21079579943077612,
 0.11215252693505033,
 0.01

In [56]:
do_math(lambda x:x**2)

[9.869604401089358,
 9.2512858703714,
 8.65296733965344,
 8.07464880893548,
 7.516330278217521,
 6.978011747499562,
 6.459693216781603,
 5.961374686063644,
 5.483056155345685,
 5.024737624627726,
 4.586419093909768,
 4.168100563191809,
 3.76978203247385,
 3.3914635017558914,
 3.033144971037933,
 2.694826440319974,
 2.3765079096020156,
 2.078189378884057,
 1.7998708481660983,
 1.5415523174481398,
 1.3032337867301813,
 1.0849152560122228,
 0.8865967252942645,
 0.708278194576306,
 0.5499596638583477,
 0.4116411331403894,
 0.29332260242243113,
 0.1950040717044729,
 0.11668554098651471,
 0.058367010268556535,
 0.020048479550598398,
 0.0017299488326403011,
 0.003411418114682239,
 0.025092887396724214,
 0.06677435667876622,
 0.12845582596080826,
 0.21013729524285035,
 0.3118187645248925,
 0.4335002338069346,
 0.5751817030889768,
 0.7368631723710191,
 0.9185446416530604,
 1.1202261109351035,
 1.341907580217147,
 1.5835890494991882,
 1.8452705187812295,
 2.126951988063273,
 2.4286334573453168,


## How else might we need to break up a report?

* How would I break a report into sentences?
* How would I break a report into words?

## Group Exercise

### Create a single string with all the reports

#### Hints, etc.
* Use List Comprehension
* Use string joins
* Iterate over the rows of the data frame
* Use TextBlob to get words